In [1]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('df.csv')

# Display the first few rows of the DataFrame to verify
print(df.head())

   ReceiverID    ActionType  NegoOutcome  \
0          32         Offer  FinalAccept   
1          31  Counteroffer  FinalAccept   
2          32  Counteroffer  FinalAccept   
3          31  Counteroffer  FinalAccept   
4          32      Question  FinalAccept   

                                             Content  Length  \
0  ['hey', 'chris', 'great', 'working', 'together...    1778   
1  ['hey', 'alex', 'pleasure', 'mine', 'starters'...    1949   
2  ['hey', 'chris', 'thank', 'response', 'glad', ...     693   
3  ['hello', 'alex', 'think', 'solid', 'compromis...     452   
4  ['dear', 'chris', 'glad', 'hear', 'willing', '...    1349   

   Word count of nego message  NegoOutcomeLabel  
0                         309                 1  
1                         337                 1  
2                         123                 1  
3                          77                 1  
4                         249                 1  


In [2]:
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import pickle
# Load the DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Tokenize the text data
def tokenize_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    return inputs

# Apply tokenization to the 'Content' column
df['Tokenized_Content'] = df['Content'].apply(tokenize_text)

# Perform inference using the model
def get_embeddings(tokenized_text):
    with torch.no_grad():
        outputs = model(**tokenized_text)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Apply the model to get embeddings for each tokenized text
df['Embeddings'] = df['Tokenized_Content'].apply(get_embeddings)


# Save the DataFrame to a pickle file
with open('df_with_embeddings.pkl', 'wb') as f:
    pickle.dump(df, f)

# Optionally, save the DataFrame to a CSV file (without embeddings)
df.drop(columns=['Tokenized_Content', 'Embeddings']).to_csv('df_with_embeddings.csv', index=False)

# Display the DataFrame with embeddings
print(df.head())

c:\Users\Alex\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [3]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Lade die DataFrame mit Embeddings aus der Pickle-Datei
with open('df_with_embeddings.pkl', 'rb') as f:
    df = pickle.load(f)

# Zielvariable (Label) und Features definieren
X = df['Embeddings'].tolist()  # Die BERT-Embeddings als Features
y = df['NegoOutcomeLabel']     # Die Zielvariable für Erfolg (1) oder keinen Erfolg (0)

# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Trainiere ein Klassifikationsmodell (z. B. RandomForest)
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Teste das Modell
y_pred = clf.predict(X_test)

# Ergebnisse auswerten
print("Model Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


c:\Users\Alex\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


Model Accuracy: 0.7915742793791575

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.10      0.18       104
           1       0.79      1.00      0.88       347

    accuracy                           0.79       451
   macro avg       0.89      0.55      0.53       451
weighted avg       0.84      0.79      0.72       451



Das Ergebnis enthält wichtige Metriken zur Bewertung des Modells, insbesondere zur Performance bei der Vorhersage der beiden Klassen (`0` und `1`). Hier ist eine Erklärung der verschiedenen Teile:

---

### **1. Model Accuracy: 0.79 (79%)**
- **Bedeutung**: Das Modell liegt in 79% der Fälle richtig, wenn man alle Vorhersagen (Klasse `0` und `1`) berücksichtigt.
- **Einschränkung**: Accuracy allein kann irreführend sein, insbesondere bei einem **unausgeglichenen Datensatz**, was hier der Fall ist (es gibt 347 Beispiele für Klasse `1` und nur 104 für Klasse `0`).

---

### **2. Classification Report**
Die wichtigsten Begriffe im Classification Report:

#### Klasse `0` (kein Erfolg):
- **Precision (1.00)**: Von allen Beispielen, die das Modell als `0` (kein Erfolg) klassifiziert hat, waren 100% tatsächlich `0`. Das Modell gibt also keine falschen Positiven für `0`.
- **Recall (0.10)**: Von allen echten Beispielen der Klasse `0` erkennt das Modell nur 10%. Es übersieht also viele Fälle, in denen es sich tatsächlich um Klasse `0` handelt.
- **F1-Score (0.18)**: Dies ist der harmonische Mittelwert von Precision und Recall. Da der Recall so niedrig ist, ist der F1-Score für Klasse `0` ebenfalls sehr niedrig.

#### Klasse `1` (Erfolg):
- **Precision (0.79)**: Von allen Beispielen, die als `1` (Erfolg) vorhergesagt wurden, waren 79% korrekt.
- **Recall (1.00)**: Von allen echten Beispielen der Klasse `1` erkennt das Modell 100%. Es übersieht keine erfolgreichen Verhandlungen.
- **F1-Score (0.88)**: Da sowohl Precision als auch Recall relativ hoch sind, ergibt sich ein guter F1-Score.

---

### **3. Durchschnittswerte**
- **Macro Avg** (Durchschnitt der Metriken über alle Klassen, ungewichtet):
  - **Precision (0.89)**: Der Durchschnitt der Precision über beide Klassen.
  - **Recall (0.55)**: Der Durchschnitt der Recall-Werte. Der niedrige Wert für Klasse `0` zieht den Durchschnitt herunter.
  - **F1-Score (0.53)**: Der Durchschnitt der F1-Scores über beide Klassen. Dies zeigt, dass das Modell für Klasse `0` schwach ist.
  
- **Weighted Avg** (Durchschnitt der Metriken, gewichtet nach der Anzahl der Beispiele pro Klasse):
  - **Precision (0.84)**: Berücksichtigt, dass die Klasse `1` häufiger vorkommt.
  - **Recall (0.79)**: Spiegelt die Gesamtaccuracy des Modells wider.
  - **F1-Score (0.72)**: Ein gewichteter F1-Score.

---

### **Interpretation**
1. **Starke Leistung für Klasse `1`** (Erfolg):
   - Das Modell erkennt alle erfolgreichen Verhandlungen (Recall = 1.00) und macht nur wenige Fehler bei der Vorhersage dieser Klasse (Precision = 0.79).
   - Dies ist hilfreich, wenn es wichtiger ist, Erfolge zuverlässig zu erkennen.

2. **Schwache Leistung für Klasse `0`** (kein Erfolg):
   - Das Modell hat große Schwierigkeiten, nicht-erfolgreiche Verhandlungen korrekt zu erkennen (Recall = 0.10). Es übersieht fast alle Fälle, in denen die Verhandlung kein Erfolg war.
   - Dies liegt vermutlich daran, dass der Datensatz unausgeglichen ist (viel mehr Beispiele für Klasse `1`).

3. **Unausgeglichener Datensatz**:
   - Die Klasse `1` dominiert den Datensatz (347 vs. 104). Dies führt dazu, dass das Modell stark auf Klasse `1` optimiert ist und Klasse `0` vernachlässigt.

---

### **Verbesserungsmöglichkeiten**
Um die Erkennung von Klasse `0` (kein Erfolg) zu verbessern:
1. **Balanciere den Datensatz aus**:
   - Verwende Techniken wie Oversampling (z. B. SMOTE) für die unterrepräsentierte Klasse `0` oder Downsampling für die überrepräsentierte Klasse `1`.

2. **Klassengewicht anpassen**:
   - Viele Modelle (z. B. Random Forest oder Logistic Regression) erlauben es, Gewichte für Klassen zu setzen. Dies hilft, das Ungleichgewicht zu kompensieren:
     ```python
     clf = RandomForestClassifier(class_weight={0: 2, 1: 1}, random_state=42)
     ```

3. **Andere Modelle ausprobieren**:
   - Modelle wie Gradient Boosting (XGBoost, LightGBM) oder neuronale Netze könnten besser mit dem Ungleichgewicht umgehen.

4. **Feature Engineering**:
   - Zusätzliche Features wie die Länge der Nachricht oder spezifische Schlüsselwörter könnten helfen, die Unterscheidung zwischen `0` und `1` zu verbessern.

---

Möchtest du eine der Verbesserungsmöglichkeiten umsetzen? Ich kann dir helfen, beispielsweise eine Methode zur Datenbalance oder zur Anpassung der Klassengewichte einzubauen!

In [4]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE  # SMOTE für Oversampling

# Lade die DataFrame mit Embeddings aus der Pickle-Datei
with open('df_with_embeddings.pkl', 'rb') as f:
    df = pickle.load(f)

# Zielvariable (Label) und Features definieren
X = df['Embeddings'].tolist()  # Die BERT-Embeddings als Features
y = df['NegoOutcomeLabel']     # Die Zielvariable für Erfolg (1) oder keinen Erfolg (0)

# Train-Test-Split (noch unausgeglichen)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Konvertiere die Features (Listen) in einen 2D-Array für SMOTE
import numpy as np
X_train = np.array(X_train)

# SMOTE anwenden, um die Klasse 0 zu oversamplen
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Optional: Überprüfe die neue Verteilung der Klassen
from collections import Counter
print("Class distribution after SMOTE:", Counter(y_train_resampled))

# Trainiere ein Klassifikationsmodell (z. B. RandomForest)
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_resampled, y_train_resampled)

# Teste das Modell
y_pred = clf.predict(X_test)

# Ergebnisse auswerten
print("Model Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


c:\Users\Alex\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


Class distribution after SMOTE: Counter({1: 1448, 0: 1448})
Model Accuracy: 0.7804878048780488

Classification Report:
               precision    recall  f1-score   support

           0       0.54      0.35      0.42       104
           1       0.82      0.91      0.86       347

    accuracy                           0.78       451
   macro avg       0.68      0.63      0.64       451
weighted avg       0.76      0.78      0.76       451



In [6]:
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# Lade die DataFrame mit Embeddings aus der Pickle-Datei
with open('df_with_embeddings.pkl', 'rb') as f:
    df = pickle.load(f)

# Zielvariable und BERT-Embeddings extrahieren
X_embeddings = df['Embeddings'].tolist()  # Die BERT-Embeddings als Features
y = df['NegoOutcomeLabel']  # Zielvariable (1 = Erfolg, 0 = Kein Erfolg)

# Feature Engineering: Zusätzliche Features aus dem Text (Content-Spalte)
df['Length'] = df['Content'].apply(len)  # Textlänge
df['Word count'] = df['Content'].apply(lambda x: len(x.split()))  # Wortanzahl


# Kombiniere alle Features: BERT-Embeddings + neue Features
X_additional = df[['Length', 'Word count'] ].values
X_embeddings = np.array(X_embeddings)
X_combined = np.hstack((X_embeddings, X_additional))  # Zusammenführen der Features

# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# SMOTE anwenden, um den Datensatz auszugleichen
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# RandomForest-Modell trainieren
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_resampled, y_train_resampled)

# Vorhersagen
y_pred = clf.predict(X_test)

# Ergebnisse auswerten
print("Model Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


c:\Users\Alex\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


Model Accuracy: 0.7649667405764967

Classification Report:
               precision    recall  f1-score   support

           0       0.49      0.33      0.39       104
           1       0.82      0.90      0.85       347

    accuracy                           0.76       451
   macro avg       0.65      0.61      0.62       451
weighted avg       0.74      0.76      0.75       451

